
# Regression mit Train-Test-Split

Dieses Notebook erweitert das einfache Beispiel: Wir prüfen, ob unser Modell auch auf neuen Daten funktioniert. Dazu teilen wir die Daten in Trainings- und Testset, trainieren eine lineare Regression und evaluieren sie mit mehreren Kennzahlen.



Ziele:

1. Aufteilen der Daten in `train` und `test`.
2. Modell nur auf `train` fitten.
3. Leistung auf `test` messen.
4. Optional: Cross-Validation für stabilere Kennzahlen.


In [ ]:

import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_validate



## 1. Daten laden

Wie zuvor nutzen wir `data/data-ads.csv` mit Werbebudget (`ads`) und Verkäufen (`sales`).


In [ ]:

df = pd.read_csv("data/data-ads.csv")
df.head()



## 2. Train-Test-Split

Wir reservieren 20 % der Daten für den abschließenden Test. `random_state` macht das Ergebnis reproduzierbar.


In [ ]:

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {train_df.shape[0]} Zeilen")
print(f"Testdaten:      {test_df.shape[0]} Zeilen")



Ein kurzer Überblick über das Trainingsset stellt sicher, dass die Verteilung plausibel aussieht.


In [ ]:

train_df.describe().T



Streudiagramm ausschließlich auf Basis der Trainingsdaten:


In [ ]:

train_scatter = (
    alt.Chart(train_df)
    .mark_circle(size=80, color="#1f77b4")
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("sales", title="Verkäufe"))
    .properties(title="Training: Werbung vs. Verkäufe")
)
train_scatter



## 3. Modell trainieren

Wir definieren Features und Zielvariable getrennt für Train und Test.


In [ ]:

feature = "ads"
target = "sales"

X_train = train_df[[feature]]
y_train = train_df[target]
X_test = test_df[[feature]]
y_test = test_df[target]



Das Modell wird ausschließlich auf den Trainingsdaten gefittet.


In [ ]:

model = LinearRegression()
model.fit(X_train, y_train)



Zur Einordnung werfen wir wieder einen Blick auf die Parameter.


In [ ]:

coef_table = pd.DataFrame(
    {
        "Parameter": ["Achsenabschnitt", "Steigung"],
        "Wert": [model.intercept_, model.coef_[0]],
    }
)
coef_table



## 4. Evaluate

Wir vergleichen Trainings- und Testergebnisse. Große Unterschiede deuten auf Overfitting hin.


In [ ]:

def evaluate_split(name, X, y):
    preds = model.predict(X)
    mse = mean_squared_error(y, preds)
    return {
        "Datensatz": name,
        "MSE": mse,
        "RMSE": mse ** 0.5,
        "R^2": r2_score(y, preds),
    }

results = [
    evaluate_split("Train", X_train, y_train),
    evaluate_split("Test", X_test, y_test),
]

pd.DataFrame(results).round(3)



Eine Grafik macht deutlich, wie gut die Vorhersagen die Testpunkte treffen.


In [ ]:

test_with_pred = test_df.assign(prediction=model.predict(X_test))
test_sorted = test_with_pred.sort_values(feature)

points = (
    alt.Chart(test_with_pred)
    .mark_circle(size=80, color="#1f77b4")
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("sales", title="Reale Verkäufe"))
)
line = (
    alt.Chart(test_sorted)
    .mark_line(color="#d62728", size=3)
    .encode(x="ads", y="prediction")
)
(points + line).properties(title="Testdaten vs. Vorhersage")



## 5. Bonus: Cross-Validation

Statt eines einzigen Splits können wir mehrere Folds verwenden. Das stabilisiert die Kennzahlen und gibt eine realistischere Spanne.


In [ ]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(
    LinearRegression(),
    X_train,
    y_train,
    cv=cv,
    scoring=("r2", "neg_mean_squared_error"),
)
summary = pd.DataFrame(
    {
        "Fold": range(1, cv_results["test_r2"].shape[0] + 1),
        "R^2": cv_results["test_r2"],
        "RMSE": (-cv_results["test_neg_mean_squared_error"]) ** 0.5,
    }
)
summary.round(3)



Zum Abschluss eine kurze Interpretation:

- Liegen Train- und Test-RMSE dicht beieinander, verallgemeinert das Modell gut.
- Cross-Validation bestätigt die Schätzungen und zeigt deren Streuung.
- Schon dieses einfache Beispiel reicht, um typische Schritte jedes Regressionsprojekts zu verstehen.
